<a href="https://colab.research.google.com/github/vinicius-vargas/robust-market-screener/blob/main/stocks_alpha_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q yfinance yahooquery setuptools pandas-datareader plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 35.0 MB/s eta 0:00:00


In [ ]:
%load_ext rpy2.ipython

In [ ]:
### Setting up libraries
import numpy as np
import pandas as pd
import yfinance as yf
import yahooquery as yq
import plotly.express as px
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import time
from scipy.stats import spearmanr
import statsmodels
import statsmodels.api as sm
from statsmodels.tools.tools import pinv_extended
from google.colab import  drive
import warnings

warnings.filterwarnings("ignore")

from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore", category=RRuntimeWarning)

### BLOCKLIST
blocklist = [
    'PETR4.SA'  ## Muita treta envolvida
    ,'VALE3.SA' ## Brumadinho e Mariana
    ,'AZUL4.SA' ## Setor ruim
    ,'GOLL4.SA' ## Setor ruim
    ,'JBSS3.SA'
    ,'AALR3.SA' ## TOP 1 Piores ESG
    ,'PNVL3.SA' ## TOP 2 Piores ESG
    ,'PNVL4.SA' ## TOP 2 Piores ESG
    ,'TRIS3.SA' ## TOP 3 Piores ESG
    ,'BRAP3.SA' ## TOP 4 Piores ESG
    ,'BRAP4.SA' ## TOP 4 Piores ESG
    ,'LAND3.SA' ## TOP 5 Piores ESG
    ,'ISAE3.SA' ## Pouco histórico
    ,'ISAE4.SA' ## Pouco histórico
]


# Adjust Cientific Notation - Importante to get correct coefficients
# pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
### Captura todas as ações negociadas do dia de hoje
assets = (
    pd.read_html( 'https://www.dadosdemercado.com.br/acoes')[0]
    .assign(
        Ticker = lambda x:x.Ticker + '.SA'
    )
    ['Ticker']
    .tolist()
)

### Calculo do Volume no Ultimo período estabelecido
start = (datetime.today() - timedelta(days=365*4)).strftime('%Y-%m-%d')
end = datetime.today().strftime('%Y-%m-%d')

assets = yf.download(assets, start = start, end = end)

### data tidying - Calculo da Volumetria por Ativo & Filtros (Volume + Duplicados)
assets = (
    assets
    .loc[:,('Volume', slice(None))]
    .droplevel(level=0, axis=1)
    [lambda x: x.index.dayofweek < 5]
    .sum()
    .reset_index(drop=False)
    .rename(columns={0:'Volume'})
    .assign(
        teste1 = (
            lambda x: (
                x.Ticker
                .str.replace('11.SA', '' )
                .str.replace('6.SA', '' )
                .str.replace('5.SA', '' )
                .str.replace('4.SA', '' )
                .str.replace('3.SA', '')
            )
        )
    )
    [lambda x: x.Volume > x.describe(percentiles=[.4, .9]).T['40%'][0]]
    .sort_values(by = ['Volume'], ascending=False)
    .drop_duplicates(subset=['teste1'], keep='first')
    ['Ticker']
    .tolist()
)

# Tickers of Global Indexes
factors = [
    '^VIX',       # Global Volatility Index
    '^IRX',       # Juros 3 meses EUA - BSHV39
    '^GSPC',      # S&P 500 - IVVB11
    'ACWX',       # MSCI - Top Ações mundo ordenado por Cap SEM USA
    'XEM.TO',     # MSCI - Ações Emergentes ordenado por Cap - BEEM39
    'EMB',        # USD Emerging Markets Bond
    'BRL=X',      # USD vs BRL
    'GD=F',       # GSCI ETF commodities - MATB11
]

# Union B3's Assets + Global Factors
assets = factors + assets

[*********************100%***********************]  398 of 398 completed
ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['AMAR11.SA', 'DASA11.SA', 'BIOM11.SA', 'AZEV11.SA', 'PINE11.SA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [ ]:
# Date Range
start = '2010-01-01'
end = datetime.today().strftime('%Y-%m-%d')

# Downloading data & adjusting it
data = yf.download(assets, start = start, end = end)
data = (
    data
    .loc[:,('Close', slice(None))]
    .droplevel(level=0, axis=1)
    [lambda x: x.index.dayofweek < 5]
    .set_index(data.index.tz_localize(None))
    .assign(
        VIX = lambda x: x['^VIX'], # * x['BRL=X']
        IRX = lambda x: x['^IRX'],
        EMB = lambda x: x['EMB'],
        GDF = lambda x: x['GD=F'],
        GSPC = lambda x: x['^GSPC'],
        ACWX = lambda x: x['ACWX'],
        XEM_TO = lambda x: x['XEM.TO']
    )
    .drop(['^VIX', '^IRX', '^GSPC', 'XEM.TO', 'GD=F'], axis=1)
    .fillna(method='ffill')
)

# Creating factor data
# factors = ['mkt', 'size', 'qld', 'momnt', 'liq']
link = 'https://nefin.com.br/resources/risk_factors/nefin_factors.csv'

factors_df = (
    pd.read_csv(link, index_col=[0])
    .rename(columns=str.lower)
    [['date', 'rm',	'smb',	'hml',	'wml', 'iml']]
    .assign(
        date = lambda x: pd.to_datetime(x['date']),
        mkt = lambda x: x['rm'],
        size = lambda x: x['smb'],
        qld = lambda x: x['hml'],
        momnt = lambda x: x['wml'],
        liq = lambda x: x['iml']
    )
    .set_index('date')
    .loc[start:end]
    .dropna()
)
factors_df = factors_df[factors_df.index.isin(data.index)]

# Join both datasets into one
data = pd.concat([data.pct_change(fill_method=None), factors_df], axis = 1)

# Turn all values from lognormal to normal
raw_data = np.log1p(data)

[*********************100%***********************]  228 of 228 completed


In [ ]:
# Factors
factors = [
    'VIX',         # Volatilidade EUA
    'IRX',         # Juros 3 meses EUA - BSHV39
    'GSPC',         # S&P 500 - IVVB11
    'ACWX',         # MSCI - Top Ações mundo ordenado por Cap SEM USA
    'XEM_TO',       # MSCI - Ações Emergentes ordenado por Cap - BEEM39
    'EMB',          # USD Emerging Markets Bond
    'GDF',          # GSCI ETF commodities - MATB11
    'BRL=X',        # USD vs BRL
    'mkt',
    'size',
    'qld',
    'momnt',
    'liq'
]

# Fix Assets List
assets = [item for item in assets if item not in ['^VIX', '^IRX', '^GSPC', 'XEM.TO', 'GD=F', 'ACWX'
]]

# Create dataframe to save alpha and betas
data = pd.DataFrame()

# Run a linear regression to get alpha
for i in tqdm([x for x in assets if x not in factors]):

  # Select index
  y = raw_data[i].dropna()

  qtd = len(y)

  # Selecting factors
  vars = raw_data[raw_data.index.isin(y.index)][factors].dropna()

  X_sm = sm.add_constant(vars[factors])

  y = y[y.index.isin(X_sm.index)]

  vol = np.std(y)

  # fit OLS model
  results = sm.OLS(y, X_sm).fit_regularized(L1_wt=0, alpha=0.0001)

  n_model = sm.OLS(y, X_sm)
  pinv_wexog,_ = pinv_extended(n_model.wexog)
  normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))

  final = sm.regression.linear_model.OLSResults(
      n_model,
      results.params,
      normalized_cov_params
  )

  a = np.where(
      i == 'GSPC', 's_p',
        np.where(
            i == '^RUT', 'rsl_2000',
              np.where(
                  i == 'EWJ' , 'top_jp',
                    np.where(
                        i == 'GDF', 'cmmdt',
                          np.where(
                              i == 'GCF', 'gld',
                                np.where(
                                    i == 'CL=F', 'oil',
                                      np.where(
                                          i == '000001.SS', 'sse_china',
                                              np.where(
                                                  i == 'IXIC', 'nsdq', i
                                                  )
                                              )
                                      )
                                )
                          )
                    )
              )
      )

  # Create the last table woth all coefficients
  dt = pd.DataFrame(
    {
     'ticker': a,
     'qtd_dias': qtd,
#     'vix': [results.params[1].round(3)],
#     'fed_3_y': [results.params[2].round(3)],
     's_p': [results.params[1].round(3)],
#     'msci_top_ex_us': [results.params[4].round(3)],
#     'msci_emg': [results.params[5].round(3)],
     'bond_emg': [results.params[2].round(3)],
     'cmmdt': [results.params[3].round(3)],
     'usd_real': [results.params[4].round(3)],
     'mkt': [results.params[5].round(3)],
     'size': [results.params[6].round(3)],
     'qld': [results.params[7].round(3)],
     'momnt': [results.params[8].round(3)],
     'liq': [results.params[9].round(3)],
     'return': [raw_data[i].sum().round(3)],
     'vol': round(vol, 5),
     'alpha': [(results.params[0]).round(5)],
     'r_score': [final.rsquared.round(3)],
     'last_update': [end]
     }
  )

  data = pd.concat([data, dt], ignore_index=True)

  0%|          | 0/220 [00:00<?, ?it/s]

In [ ]:
### Plot Avaliando - O indicador de comportamental passado prediz o futuro?
var = 'alpha'
df_plot = data[[var, 'return']].dropna()
corr, _ = spearmanr(df_plot[var], df_plot['return'])

fig = px.scatter(
    data, x=var, y='return', hover_name='ticker',
    labels={
        var: 'Exposição ao Fator',
        'return': 'Retorno'
    },
    title=f'Relação Exposição ao Fator (Alpha) & Retorno das Ações do Ibovespa',
    trendline='ols',
    trendline_color_override = 'black',
    template='plotly_white'
).update_traces(
    marker_size=12,
    marker=dict(color='green'),
    opacity=0.4
).update_layout(
    font=dict(size=14), showlegend=False
)

fig.show()

In [ ]:
#%%R
#install.packages('cutpointr')

In [ ]:
#%%R

#library(cutpointr)
#library(tidyverse)
#library(readxl)

#data <- read_csv('teste1.csv', show_col_types = FALSE)

#data <- data %>%
#  mutate(flag_return = if_else(return > 0.1, 1, 0))

# Determinação do ponto de corte ótimo
#best_cut <- cutpointr(
#  data = data,
#  x = momnt,
#  class = flag_return,
#  pos_class = 1,
#  direction = '>=',
#  method = maximize_metric,
#  metric = accuracy
#)

# Exibir resumo das métricas c/ score otimizado
#best_cut[, c(2, 4, 8)]

In [ ]:
### Remoção da BLOCKLIST
data = data[(~data['ticker'].isin(blocklist))]

### Filtros de Exposição à Fatores
resumo = data.describe(percentiles=[.02, .98]).T
resumo = resumo.iloc[:, 4:7]

resumo.columns = ['lower', 'mid', 'upper']

### Filtro final - Relativizado
final_data = (
    data[
          # Maior, mais retorno
          (data['qtd_dias'] >= resumo['mid'].loc['qtd_dias'] * 0.75)

          # Maior, mais retorno
          & (data['s_p'] >= resumo['lower'].loc['s_p'])

          & (data['bond_emg'] >= resumo['lower'].loc['bond_emg'])
          & (data['bond_emg'] <= resumo['upper'].loc['bond_emg'])

          # Menor, mais retorno
          & (data['usd_real'] >= resumo['lower'].loc['usd_real'])
          & (data['usd_real'] <= resumo['upper'].loc['usd_real'])

          & (data['mkt'] <= resumo['upper'].loc['mkt'])

          & (data['size'] <= resumo['upper'].loc['size'])

          & (data['qld'] >= resumo['lower'].loc['qld'])

          & (data['momnt'] >= resumo['lower'].loc['momnt'])

          & (data['liq'] >= resumo['lower'].loc['liq'])
          & (data['liq'] <= resumo['upper'].loc['liq'])

          & (data['vol'] <= resumo['upper'].loc['vol'])

          & (data['return'] >= resumo['mid'].loc['return'] * 1.5)

          & (data['alpha'] > 0)
      ]
      .sort_values(by=['alpha'], ascending=False)
      .reset_index(drop=True)
)

final_data

,ticker,qtd_dias,s_p,bond_emg,cmmdt,usd_real,mkt,size,qld,momnt,liq,return,vol,alpha,r_score,last_update
0,STBP3.SA,3918,0.013,0.002,0.075,0.066,0.052,0.112,-0.010,-0.019,0.404,3.912,0.02874,0.00082,0.125,2025-01-13
1,WEGE3.SA,3918,-0.019,-0.000,0.047,0.063,0.101,0.069,0.033,0.025,0.392,3.272,0.01964,0.00069,0.263,2025-01-13
2,RADL3.SA,3918,-0.022,-0.000,0.013,0.016,0.070,0.038,-0.028,-0.010,0.348,2.604,0.01996,0.00063,0.167,2025-01-13
3,SAPR4.SA,3918,0.002,0.000,0.056,0.032,0.036,0.084,-0.015,-0.041,0.360,2.942,0.02224,0.00062,0.146,2025-01-13
4,FRAS3.SA,3918,-0.001,-0.001,0.046,-0.013,0.009,0.065,0.024,0.006,0.281,2.505,0.03995,0.00057,0.029,2025-01-13
5,UNIP6.SA,3918,-0.009,0.003,0.088,0.073,0.071,0.049,0.083,-0.016,0.371,2.917,0.02532,0.00057,0.153,2025-01-13
6,SHUL4.SA,3918,-0.001,0.004,0.041,0.043,0.031,0.070,0.033,-0.019,0.340,2.421,0.02359,0.00054,0.132,2025-01-13
7,RANI3.SA,3918,0.009,0.002,0.089,0.065,0.069,0.068,0.007,0.021,0.255,2.469,0.03107,0.00054,0.047,2025-01-13
8,SLCE3.SA,3918,-0.009,-0.003,0.043,0.055,0.051,-0.003,0.099,0.002,0.257,2.442,0.02280,0.00052,0.104,2025-01-13
9,EQTL3.SA,3918,-0.009,0.002,0.030,0.013,0.056,0.071,-0.024,0.004,0.379,2.392,0.01881,0.00050,0.202,2025-01-13


In [ ]:
### Save the output inside Google Drive
# drive.mount('drive')

# final_data.to_csv('/content/drive/My Drive/data_lake/alpha_raking.csv', encoding='utf-8', index=False)

In [ ]:
# Select tickers to get fundamentalist informations #'DEXP3.SA'
lista = final_data['ticker']

# Create dataframe to save fundamental indexes
data = pd.DataFrame()

for ticker in tqdm(lista):
  ### Get the Historical Company Performance - Gross and Net Margin
  #################################################################
  yf_data = yq.Ticker(ticker)
  asst_data = yf_data.history(period = '10y').reset_index(0)
  asst_data['year'] = pd.to_datetime(asst_data.index.to_series(), errors='coerce', utc=True).dt.year

  if 'dividends' not in asst_data.columns:
    asst_data['dividends'] = 0

  ### Get the last price of each year
  time.sleep(0.1)
  last_prices = asst_data.groupby('year')['close'].agg(['last'])

  ### Get the Historical Dividend Yield
  #####################################
  sun_div = asst_data[asst_data.dividends != 0].groupby('year')['dividends'].agg(['sum'])

  ### Grouping Last Price with Dividends Sum
  asst_div_data = pd.concat([last_prices, sun_div], axis=1)

  asst_div_data['yield'] = asst_div_data['sum'] / asst_div_data['last']

  hist_div = round(asst_div_data['yield'].median() * 100, 2)


  ### Final Dataset - Fundamentalist Performance & Index
  ######################################################

  final_data_fund = pd.DataFrame(
      {
      'ticker': ticker,
      'Div. Yield Med': [hist_div],
      }
  )

  time.sleep(0.1)

  data = pd.concat([data, final_data_fund], ignore_index=True)

  0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
final_data = final_data.merge(data, on='ticker', how='left')[(lambda x: x['Div. Yield Med'] >= 3)]

final_data.head(59)

,ticker,qtd_dias,s_p,bond_emg,cmmdt,usd_real,mkt,size,qld,momnt,liq,return,vol,alpha,r_score,last_update,Div. Yield Med
0,STBP3.SA,3918,0.013,0.002,0.075,0.066,0.052,0.112,-0.010,-0.019,0.404,3.912,0.02874,0.00082,0.125,2025-01-13,3.06
3,SAPR4.SA,3918,0.002,0.000,0.056,0.032,0.036,0.084,-0.015,-0.041,0.360,2.942,0.02224,0.00062,0.146,2025-01-13,5.23
4,FRAS3.SA,3918,-0.001,-0.001,0.046,-0.013,0.009,0.065,0.024,0.006,0.281,2.505,0.03995,0.00057,0.029,2025-01-13,3.33
5,UNIP6.SA,3918,-0.009,0.003,0.088,0.073,0.071,0.049,0.083,-0.016,0.371,2.917,0.02532,0.00057,0.153,2025-01-13,4.53
6,SHUL4.SA,3918,-0.001,0.004,0.041,0.043,0.031,0.070,0.033,-0.019,0.340,2.421,0.02359,0.00054,0.132,2025-01-13,5.05
7,RANI3.SA,3918,0.009,0.002,0.089,0.065,0.069,0.068,0.007,0.021,0.255,2.469,0.03107,0.00054,0.047,2025-01-13,4.35
8,SLCE3.SA,3918,-0.009,-0.003,0.043,0.055,0.051,-0.003,0.099,0.002,0.257,2.442,0.02280,0.00052,0.104,2025-01-13,4.44
10,PSSA3.SA,3918,-0.009,-0.002,0.035,0.041,0.087,0.020,-0.015,-0.050,0.324,2.053,0.01834,0.00046,0.182,2025-01-13,4.93
12,BBSE3.SA,3053,-0.009,0.000,0.010,0.050,0.099,0.006,-0.002,-0.016,0.451,1.597,0.01793,0.00044,0.339,2025-01-13,6.39
13,FESA4.SA,3918,-0.021,0.001,0.044,0.069,0.065,0.010,0.100,-0.034,0.353,1.783,0.02448,0.00040,0.168,2025-01-13,6.16


In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

corr = raw_data[final_data.ticker].corr(method='spearman')
mask = np.triu(np.ones_like(corr, dtype=np.bool_))
corr = corr.mask(mask)

fig = ff.create_annotated_heatmap(
    z=corr.to_numpy().round(2),
    x=list(corr.index.values),
    y=list(corr.columns.values),
    xgap=3, ygap=3,
    zmin=-0.5, zmax=1,
    colorscale='earth',
    colorbar_thickness=30,
    colorbar_ticklen=3
)

fig.update_layout(
    width=1200, height=1200,
    xaxis_showgrid=False,
    xaxis={'side': 'bottom'},
    yaxis_showgrid=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

fig.show()

In [ ]:
# Tickers of Global Indexes
assets = final_data['ticker']

# Load all data
data = pd.DataFrame()

for i in assets:
  df = (
      yq.Ticker(i)
      .history(start = start, end = end, interval = '1d')
      .reset_index(0)
      [lambda x: pd.to_datetime(x.index).dayofweek < 5]
      [['adjclose', 'dividends']]
      .fillna(method='ffill')
  )

  data[i] = df['adjclose']
  data['div_' + i] = df['dividends']


data.index = pd.to_datetime(data.index).tz_localize('UTC')
data = data.fillna(method='ffill')


### Reinvestimentos
amount_asset = 1000000 / len(assets)

for c in data[assets]:

  data['qtd_pst_' + c] = round(amount_asset / data[c].head(1), 0)

  for i in range(1, len(data)):

    if data['div_' + c][i-90] > 0:
      data['qtd_pst_' + c][i] = round(
          data['qtd_pst_' + c][i-1]
          + (data['div_' + c][i-90] * data['qtd_pst_' + c][i-1] * 0.9 / data[c][i])
          , 0
        )
    else:
      data['qtd_pst_' + c][i] = data['qtd_pst_' + c][i-1]

  data['value_' + c] = data['qtd_pst_' + c] * data[c]


div_data = data[data.filter(like='value').columns]
div_data.index = pd.to_datetime(div_data.index)
div_data.index = div_data.index.tz_convert('UTC')

div_data.pct_change().sum().sort_values(ascending=False)

,0
value_STBP3.SA,15.088743
value_SYNE3.SA,7.627217
value_DEXP3.SA,6.883225
value_FRAS3.SA,6.522061
value_CMIG4.SA,6.171652
value_UNIP6.SA,5.743813
value_SAPR4.SA,5.516678
value_RANI3.SA,5.079740
value_SHUL4.SA,4.603999
value_SLCE3.SA,4.273552


In [ ]:
assets = final_data['ticker']

# Load all data
data = pd.DataFrame()

for i in assets:
  df = (
      yq.Ticker(i)
      .history(start = start, end = end, interval = '1d')
      .reset_index(0)
      [lambda x: pd.to_datetime(x.index).dayofweek < 5]
      [['adjclose', 'dividends']]
      .fillna(method='ffill')
  )

  data[i] = df['adjclose']
  data['div_' + i] = df['dividends']


data.index = pd.to_datetime(data.index).tz_localize('UTC')
data = data.fillna(method='ffill')


### Reinvestimentos
amount_asset = 1000000 / len(assets)

for c in data[assets]:

  data['qtd_pst_' + c] = round(amount_asset / data[c].head(1), 0)

  for i in range(1, len(data)):

    if data['div_' + c][i-1] > 0:
      data['qtd_pst_' + c][i] = round(
          data['qtd_pst_' + c][i-1]
          + (data['div_' + c][i-1] * data['qtd_pst_' + c][i-1] * 0 / data[c][i])
          , 0
        )
    else:
      data['qtd_pst_' + c][i] = data['qtd_pst_' + c][i-1]

  data['value_' + c] = data['qtd_pst_' + c] * data[c]


div_data = data[data.filter(like='value').columns]
div_data.index = pd.to_datetime(div_data.index)
div_data.index = div_data.index.tz_convert('UTC')

div_data.pct_change().sum().sort_values(ascending=False)

,0
value_DEXP3.SA,6.645895
value_FRAS3.SA,5.810675
value_STBP3.SA,5.736671
value_RANI3.SA,4.288608
value_UNIP6.SA,4.117169
value_SAPR4.SA,3.869499
value_SHUL4.SA,3.456601
value_SLCE3.SA,3.415909
value_KEPL3.SA,3.242648
value_FESA4.SA,2.910728
